In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lupus_(Systemic_Lupus_Erythematosus)"
cohort = "GSE154851"

# Input paths
in_trait_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)"
in_cohort_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)/GSE154851"

# Output paths
out_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/GSE154851.csv"
out_gene_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE154851.csv"
out_clinical_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/clinical_data/GSE154851.csv"
json_path = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Investigation Of Genes Associated With Atherosclerosis In Patients With Systemic Lupus Erythematosus"
!Series_summary	"Systemic lupus erythematosus (SLE) is a chronic, autoimmune disease affecting multiple heterogeneous organs and systems. SLE is associated with increased risk of atherosclerosis and increased cardiovascular complications. In this study, we specifically aimed to identify patients with SLE who are genetically at risk for developing atherosclerosis. Sureprint G3 Human Gene Expression 8x60K Microarray kit (Agilent technologies, Santa Clara, CA, USA) was used in our study. Genes showing differences in expression between the groups were identified by using GeneSpring GX 10.0 program. A total of 155 genes showing expression level difference were detected between SLE patients and healthy controls. In molecular network analysis."
!Series_overall_design	"38 patients with systemic lupus erythematosus (36 females, 2 males) and 32 healthy cont

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Check gene expression data availability
import os
import pandas as pd
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# From the background information, we can see that this dataset used 
# "Sureprint G3 Human Gene Expression 8x60K Microarray kit" which indicates gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1. Trait (SLE)
# From the sample characteristics dictionary, we don't see explicit trait information
# The trait isn't directly available in the sample characteristics
trait_row = None

def convert_trait(trait_str):
    if trait_str is None:
        return None
    if ':' in trait_str:
        trait_value = trait_str.split(':', 1)[1].strip().lower()
        # Assuming SLE patients are coded as 1, healthy controls as 0
        if any(term in trait_value for term in ['sle', 'lupus', 'patient', 'case', 'disease']):
            return 1
        elif any(term in trait_value for term in ['control', 'healthy', 'normal']):
            return 0
    return None

# 2.2. Age
# Age is available in the sample characteristics dictionary at key 2
age_row = 2

def convert_age(age_str):
    try:
        if age_str is None:
            return None
        # Extract the age value after the colon
        if ':' in age_str:
            age_part = age_str.split(':', 1)[1].strip()
            # Remove 'y' and convert to float
            return float(age_part.replace('y', '').strip())
        return None
    except:
        return None

# 2.3. Gender
# Gender is available in the sample characteristics dictionary at key 1
gender_row = 1

def convert_gender(gender_str):
    if gender_str is None:
        return None
    if ':' in gender_str:
        gender = gender_str.split(':', 1)[1].strip().lower()
        if gender == 'female':
            return 0
        elif gender == 'male':
            return 1
    return None

# 3. Save Metadata
# Trait data is not available in the sample characteristics
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we should skip the clinical feature extraction step
if trait_row is not None:
    # This block won't execute but is kept for completeness
    clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.csv"))
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 62976
First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Gene Identifier Review
# The identifiers shown are simply numbers (1, 2, 3, etc.) which are clearly not human gene symbols.
# These appear to be probe indices or database-specific identifiers that would need to be mapped
# to proper human gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.642618', 'Hs.348522'], 'ENSEMBL_ID': [nan, nan, nan, 'ENST00000014930', 'ENST00000281830'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788'], 'CHROMOSOMAL_LOCATION': 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation dataframe correspond to probe IDs and gene symbols
prob_col = 'ID'  # Probe identifiers in gene expression data match the 'ID' column
gene_col = 'GENE_SYMBOL'  # Gene symbols are stored in the 'GENE_SYMBOL' column

# 2. Create a gene mapping dataframe using these two columns
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created mapping dataframe with {len(mapping_df)} entries")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"\nConverted to gene expression data with {len(gene_data)} genes")
print("First few gene symbols:")
print(gene_data.index[:10])

# Apply normalization to gene symbols to handle potential synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nAfter normalizing gene symbols, we have {len(gene_data)} unique genes")
print("First few normalized gene symbols:")
print(gene_data.index[:10])


Created mapping dataframe with 54295 entries
First few rows of mapping dataframe:
  ID          Gene
3  4         HEBP1
4  5         KCNE4
5  6        BPIFA3
6  7  LOC100129869
7  8          IRG1

Converted to gene expression data with 20353 genes
First few gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT',
       'A4GNT', 'AA06'],
      dtype='object', name='Gene')



After normalizing gene symbols, we have 19847 unique genes
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT',
       'AA06', 'AAA1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Based on our analysis in Step 2, we determined that this dataset does not contain 
# lupus trait data. The dataset is about recurrent pregnancy losses and unexplained infertility,
# not Lupus (Systemic Lupus Erythematosus).

# Create a dummy clinical dataframe to properly document the absence of trait data
sample_ids = normalized_gene_data.columns.tolist()
clinical_df = pd.DataFrame(index=sample_ids)
# We'll create an empty clinical dataframe to accurately represent that we don't have lupus data
print(f"Clinical data not available for {trait} in this dataset")

# 3. Prepare linked data - in this case, it will just be the gene expression data
# with missing clinical information
linked_data = normalized_gene_data.T
print(f"Shape of linked data: {linked_data.shape}")

# 4. We cannot properly handle missing values for trait because the trait data 
# doesn't exist in this dataset

# 5. Since the trait data is completely missing, we shouldn't proceed with bias analysis
is_trait_biased = True  # Mark as biased because we have no trait data for lupus
unbiased_linked_data = linked_data  # Keep the gene data for reference

# 6. Conduct quality check and save the cohort information - report that trait data is not available
is_trait_available = False  # No trait data for lupus
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from endometrial tissue study of recurrent pregnancy losses " +
         "and unexplained infertility, not lupus. No lupus trait data available."
)

# 7. Since the linked data is not usable for lupus analysis, we won't save it as a final dataset
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed: No lupus trait data available. Final linked data not saved.")

Top 10 gene indices before normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06', 'AAA1']
Top 10 gene indices after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06', 'AAA1']
Shape of normalized gene data: (19847, 70)


Saved normalized gene data to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE154851.csv
Clinical data not available for Lupus_(Systemic_Lupus_Erythematosus) in this dataset
Shape of linked data: (70, 19847)
Dataset validation failed: No lupus trait data available. Final linked data not saved.
